In [29]:
#Calculate nutritional value of an order

import pandas as pd

# Define a function to calculate the nutritional value of an order
def calculate_nutritional_value(order):
    high = 0
    medium = 0
    low = 0
    for item in order:
        if item in nutritional_values:
            nutritional_value = nutritional_values[item]
            if nutritional_value == "High":
                high += 1
            elif nutritional_value == "Medium":
                medium += 1
            elif nutritional_value == "Low":
                low += 1
    return high, medium, low

# Define a function to classify an order based on its nutritional value
def classify_order(order):
    high, medium, low = calculate_nutritional_value(order)
    if high > medium > low or high > low > medium:
        return "Optimal"
    elif medium > high > low or medium > low > high or high == medium or high == low:
        return "SubOptimal"
    elif low > medium > high or low == medium or low > high > medium:
        return "Poor"

# Load the data from the Excel file
orders = pd.read_excel("DataModel.xlsx", sheet_name="TopItems")

# Load the item nutritional values from the Items sheet
items = pd.read_excel("DataModel.xlsx", sheet_name="Items")

# Create a dictionary to map item names to their nutritional values
nutritional_values = dict(zip(items["Item"], items["NutritionalValue"]))

# Iterate over each order
for index, row in orders.iterrows():
    order_items = row["TopItems"].split(", ")
    # Perform the desired calculations or analysis on the order
    # For example, you can call the "calculate_nutritional_value" or "classify_order" function from my previous answer:
    high, medium, low = calculate_nutritional_value(order_items)
    classification = classify_order(order_items)
    pantryId = row["PantryID"]
    #print(f"{pantryId}: High={high}, Medium={medium}, Low={low}, Classification={classification}")
    orders.at[index, "High"] = high
    orders.at[index, "Medium"] = medium
    orders.at[index, "Low"] = low
    orders.at[index, "NutritionalValueClassification"] = classification
print(orders)
orders.to_excel("DataModel.xlsx", sheet_name="TopItems", index=False)


    PantryID                                           TopItems  High  Medium  \
0   AFCBP776  Pudding cups, Boxed stuffing mix, Hot cocoa mi...   2.0    10.0   
1   AFCBP535  Chips, Cereal, Jelly or jam, Nuts, Oatmeal, Je...   7.0     6.0   
2   AFCBP473  Pancake mix and syrup, Chips, Canned tuna or c...   5.0     7.0   
3   AFCBP951  Dried fruit, Canned vegetables, Tea bags, Boxe...   6.0     7.0   
4   AFCBP278  Canned spaghetti or ravioli, Mustard, Boxed st...   3.0     7.0   
5   AFCBP308  Baking powder, Dried fruit, Ketchup, Chips, To...   3.0     9.0   
6   AFCBP649  Mayonnaise, Condensed milk, Boxed stuffing mix...   5.0     4.0   
7   AFCBP206  Cooking oil, Canned spaghetti or ravioli, Trai...   3.0     9.0   
8   AFCBP309  Jello cups, Pancake mix and syrup, Flour, Hot ...   5.0     6.0   
9   AFCBP108  Crackers, Cooking oil, Hot cocoa mix, Flour, G...   7.0     6.0   
10  AFCBP800  Shelf-stable milk, Boxed potatoes, Spices, May...   4.0     7.0   
11  AFCBP853  Spices, Sugar,

In [1]:
#Find Cultural Relevance of an Order

import pandas as pd

# Load the data from the Excel file
top_items = pd.read_excel("DataModel.xlsx", sheet_name="TopItems")
pantries = pd.read_excel("DataModel.xlsx", sheet_name="Pantry")
cultural_food = pd.read_excel("DataModel.xlsx", sheet_name="CulturalFood")

# Create a dictionary to map ethnic groups to their food preferences
ethnic_food_preferences = {}
for index, row in cultural_food.iterrows():
    ethnic_group = row["Ethnic Group"]
    food_preference = row["Food Preference"]
    if ethnic_group in ethnic_food_preferences:
        ethnic_food_preferences[ethnic_group].append(food_preference)
    else:
        ethnic_food_preferences[ethnic_group] = [food_preference]

# Iterate over each pantry
for index, row in pantries.iterrows():
    pantry_id = row["PantryID"]
    ethnic_groups = row["Ethnic Group"].split(", ")
    matching_items = set()
    for ethnic_group in ethnic_groups:
        #print(pantry_id,' ',ethnic_group)
        if ethnic_group in ethnic_food_preferences:
            #print(ethnic_food_preferences[ethnic_group])
            matching_items.update(set(ethnic_food_preferences[ethnic_group]))
    #print(matching_items)
    #print(top_items.loc[top_items["PantryID"] == pantry_id, "TopItems"].tolist()[0].split(", "))
    matching_items_string = ", ".join(matching_items)
    count = sum([1 for item in top_items.loc[top_items["PantryID"] == pantry_id, "TopItems"].tolist()[0].split(", ") if item in matching_items_string])
    cultural_relevance = "Yes" if count >= 7 else "No"
    top_items.loc[top_items["PantryID"] == pantry_id, "CulturalRelevance"] = cultural_relevance

# Save the updated data to the Excel file
with pd.ExcelWriter("DataModel.xlsx") as writer:
    top_items.to_excel(writer, sheet_name="TopItems", index=False


In [ ]:
##Create clusters for K-Nearest Neighbours (K=3) for all Pantries

import pandas as pd
import math

RADIUS = 6371  # km

def distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return RADIUS * c

def find_nearest_neighbors(df, index, n):
    distances = []
    for i, row in df.iterrows():
        if i == index:
            continue
        dist = distance(df.at[index, 'Latitude'], df.at[index, 'Longitude'], row['Latitude'], row['Longitude'])
        #print(df.at[index, 'PantryID'])
        distances.append((i, dist))
    distances.sort(key=lambda x: x[1])
    #print([df.at[x[0], 'PantryID'] for x in distances[:n]])
    return [df.at[x[0], 'PantryID'] for x in distances[:n]]

# Load DataModel.xlsx
df = pd.read_excel('DataModel.xlsx', sheet_name='Pantry')

# Create a new dataframe for the clusters
cluster_df = pd.DataFrame(columns=['ClusterID', 'ClusterGroup'])

# Iterate over each pantry and find its three nearest neighbors
for i, row in df.iterrows():
    neighbors = find_nearest_neighbors(df, i, 3)
    #print(df.at[i, 'PantryID'])
    neighbors_str = ', '.join([x for x in neighbors])
    cluster_str = neighbors_str +', ' + row['PantryID']
    random_number = random.randint(1000, 5000)
    clusterId = 'AFCBC' + str(random_number)
    cluster_row = {'ClusterID': clusterId, 'ClusterGroup': cluster_str}
    cluster_df = cluster_df.append(cluster_row, ignore_index=True)
    #cluster_df = cluster_df.append({'ClusterID': row['PantryID'], 'ClusterGroup': neighbors + df.at[i, 'PantryID']}, ignore_index=True)

# Save the cluster dataframe to a new sheet in DataModel.xlsx
with pd.ExcelWriter('DataModel.xlsx', mode='a') as writer:
    cluster_df.to_excel(writer, sheet_name='Cluster', index=False)

In [ ]:
#Food prediction for a pantry

import pandas as pd

# Load the data from FoodOrders sheet in DataModel.xlsx file
df = pd.read_excel('DataModel.xlsx', sheet_name='FoodOrders')

# Convert Order_Date column to datetime format
df['Order_Date'] = pd.to_datetime(df['Order_Date'])

# Filter the data for May in the past two years
start_date = pd.Timestamp('now').normalize().to_period('M').to_timestamp()
end_date = start_date - pd.DateOffset(years=2)
filtered_df = df[(df['Order_Date'].dt.month == 5) & (df['Order_Date'] >= end_date) & (df['Order_Date'] < start_date)]

total_quantity = filtered_df.groupby('PantryID').agg(
    total_quant=('Quantity', 'sum'),
    order_count=('PantryID', 'count'),
    mean_quant=('Quantity', 'mean')
)

total_quantity['mean_quant'] = total_quantity['mean_quant'].astype(int)

# Create an empty dataframe to store the predicted values
predict_df = pd.DataFrame(columns=['PantryID', 'PredictedCount'])

# Loop through the rows of total_quantity dataframe and append the predicted values to predict_df
for i, row in total_quantity.iterrows():
    predict_row = {'PantryID': row.name, 'PredictedCount': row['mean_quant']}
    predict_df = predict_df.append(predict_row, ignore_index=True)

# Save the predicted dataframe to a new sheet in DataModel.xlsx
with pd.ExcelWriter('DataModel.xlsx', mode='a') as writer:
    predict_df.to_excel(writer, sheet_name='FoodPrediction', index=False)
